In [3]:
import os
from pydub import AudioSegment
import librosa
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# 将 m4a 格式转换为 wav 格式
def formatConvert(audio_path, audio_format):
    audio_name = os.path.splitext(audio_path)[0]
    audio = AudioSegment.from_file(audio_path, format=audio_format)
    audio.export(audio_name + ".wav", format="wav")

In [5]:
formatConvert("audio/shop.m4a", "m4a")

预加重的好处：

1. 语音信号的高频部分（如辅音）通常比低频部分（如元音）能量弱。

2. 高频部分包含许多重要的语音特征信息。

3. 预加重可以提升高频分量，使整个频谱更加平坦，便于后续分析。

预加重的原理是一个简单的高通滤波器：

$$
H(z) = 1 - \alpha z^{-1}
$$

假设原输入信号为 $x[n]$，预加重后的信号为 $y[n]$，则有：

$$
Y(z) = H(z)X(z) = (1 - \alpha z^{-1})X(z) = X(z) - \alpha z^{-1}X(z)
$$

那么，

$$
y[n] = x[n] - \alpha x[n-1]
$$

In [6]:
# 预加重
def preEmphasis(x, alpha=0.97):
    return np.append(x[0], x[1:] - alpha * x[:-1])

In [7]:
def FFT(x):

    x = np.asarray(x, dtype=complex)
    N = len(x)
    assert N > 0 and (N & (N - 1)) == 0, "N must be a power of 2"
    
    if N <= 1:
        return x
    
    # X(k) = G(k) + e^(-2πk/N) H(k)
    G = FFT(x[0::2])     # g(r) = x(2r)
    H = FFT(x[1::2])      # h(r) = x(2r+1)
    
    # 旋转因子列表
    # W1, W2, ..., W(N/2-1)
    W = np.exp(-2j * np.pi * np.arange(N // 2) / N)
    
    # 
    X = np.zeros(N, dtype=complex)
    X[:N//2] = G + W * H
    X[N//2:] = G - W * H
    
    return X

In [8]:
# 计算 STFT
# y: 音频信号
# sr: 采样率
# win: 窗口大小
# step: 步长
def STFT(y, sr, win, step):
    # 分帧
    frames = []
    for i in range(0, len(y) - win, step):
        frame = np.hanning(win) * y[i:i + win]
        frames.append(frame)
    
    # 计算 STFT
    stft = []
    for frame in frames:
        # 计算 FFT
        fft = FFT(frame)
        stft.append(np.abs(fft[:win // 2 + 1])) # 幅值取绝对值，只关心正频率
    
    # 转置为 (频率, 时间) 形式
    return np.array(stft).T

Mel 滤波器是一组三角形带通滤波器，排列在 Mel 频率刻度上，用于将线性频谱转换为（符合人耳感知的）Mel 频谱。

- 数量：通常 20-40 个（常用26个）

- ​分布：在 Mel 频率刻度上非均匀分布（低频密集，高频稀疏）

- ​形状：三角形，相邻滤波器有 50% 的重叠区域

In [9]:
def hz2mel(hz):
    return 2595 * np.log10(1 + hz / 700)

def mel2hz(mel):
    return 700 * (10**(mel / 2595) - 1)

In [10]:
# Mel 滤波器组
def melFilterBank(sr, n_fft, n_filter):
    min_hz = 0
    max_hz = sr / 2

    min_mel = hz2mel(min_hz)
    max_mel = hz2mel(max_hz)

    # 在梅尔刻度上均匀分布
    mel_points = np.linspace(min_mel, max_mel, n_filter + 2)
    hz_points = mel2hz(mel_points)

    indices = np.floor((n_fft + 1) * hz_points / sr).astype(int)    # 在 FFT 结果数组中的索引

    # 空滤波器组
    filterbank = np.zeros((n_filter, n_fft // 2 + 1))

    # 构建每一个滤波器
    for i in range(n_filter):
        left = indices[i]
        center = indices[i+1]
        right = indices[i+2]

        # 三角形滤波器
        for j in range(left, center):
            filterbank[i][j] = (j - left) / (center - left)
        for j in range(center, right):
            filterbank[i][j] = (right - j) / (right - center)

    return filterbank

In [11]:
# 计算 MFCC
# y: 音频信号
# sr: 采样率
# N: 窗口大小
# D: MFCC系数数量，通常为 12~16
# M: Mel 滤波器组数量，通常为 20~40
def computeMFCC(y, sr, N, D=13, M=26):
    mfcc = np.zeros((D, log_mel_spec.shape[1])) # (系数数量, 窗口数量)

    # 预加重
    y_ = preEmphasis(y)

    # 分帧 + STFT
    spec = STFT(y_, sr, N, N//2)                # (频率, 窗口)

    # 转换为功率谱
    power_spec = np.square(spec) / N            # (频率, 窗口)

    # 应用 Mel 滤波器组
    filter_bank = melFilterBank(sr, N, M)
    mel_spec = np.dot(filter_bank, power_spec)  # (滤波器数量, 窗口)

    # 离散余弦变换(DCT)
    log_mel_spec = np.log(mel_spec + 1e-10)     # 取对数
    for i in range(D):
        for j in range(M):
            mfcc[i] += log_mel_spec[j] * np.cos(np.pi * (j - 0.5) * i / M)

    return mfcc

In [ ]:
sample_rate = 8000
y, _ = librosa.load("audio/shop.wav", sr=sample_rate)